In [23]:
!pip install cvxopt

In [12]:
import cvxopt
import numpy as np

In [2]:
from sklearn.metrics import accuracy_score

In [19]:
from sklearn.datasets import load_breast_cancer
X,y=load_breast_cancer().data,load_breast_cancer().target

In [20]:
from sklearn.model_selection import train_test_split
X,X_test,y,y_test=train_test_split(X,y,test_size=0.2)

In [21]:
y[y==0]=-1
y_test[y_test==0]=-1

In [22]:
def linear(x, z):
    return np.dot(x, z.T)


def polynomial(x, z, p=5):
    return (1 + np.dot(x, z.T)) ** p


def gaussian(x, z, sigma=0.3):
    return np.exp(-np.linalg.norm(x - z, axis=1) ** 2 / (2 * (sigma ** 2)))

def rbf_kernel(x1,x,gamma, **kwargs):
    def f(x1, x2):
        distance = np.linalg.norm(x1 - x2) ** 2
        return np.exp(-gamma * distance)
    return f


class SVM:
    def __init__(self, kernel=gaussian, C=1,gamma=1):
        self.kernel = gaussian
        self.C = C
        self.gamma=gamma

    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape

        # Calculate Kernel
        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i, :] = self.kernel(X[i, np.newaxis], self.X)

        # Solve with cvxopt final QP needs to be reformulated
        # to match the input form for cvxopt.solvers.qp
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        q = cvxopt.matrix(-np.ones((m, 1)))
        G = cvxopt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt.matrix(y, (1, m), "d")
        b = cvxopt.matrix(np.zeros(1))
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol["x"])

    def predict(self, X):
        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)

        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv]
                * self.y[sv, np.newaxis]
                * self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )

        return np.sign(y_predict + self.b)

    def get_parameters(self, alphas):
        threshold = 1e-7

        sv = ((alphas > threshold) * (alphas < self.C)).flatten()
        self.w = np.dot(X[sv].T, alphas[sv] * self.y[sv, np.newaxis])
        self.b = np.mean(
            self.y[sv, np.newaxis]
            - self.alphas[sv] * self.y[sv, np.newaxis] * self.K[sv, sv][:, np.newaxis]
        )
        return sv


if __name__ == "__main__":
    np.random.seed(1)
  

    svm = SVM(kernel=polynomial)
    svm.fit(X, y)
    y_pred = svm.predict(X_test)
    y_pred_train=svm.predict(X)
    print(f"Accuracy: {sum(y==y_pred_train)/y.shape[0]}")

    print(f"Accuracy: {sum(y_test==y_pred)/y_test.shape[0]}")

Accuracy: 1.0
Accuracy: 0.631578947368421
